# make_dataset

ダウンロードしたスレッドタイトルをmecab＋neologd辞書で形態素解析し、トークンID列としてまとめてpickleに固める。

|パラメータ名|内容|
|:--|:--|
| `max_vocab` | トークンIDの最大ボキャブラリ数 |
| `THREAD_FILTER_TH_MIN_RESPONSE` | スレのフィルタリング条件：最低レス数 |
| `THREAD_FILTER_TH_MIN_TOKEN` | スレのフィルタリング条件：最低トークン数 |
| `THREAD_FILTER_TH_MAX_TOKEN` | スレのフィルタリング条件：最大トークン数 |
| `THREAD_FILTER_TH_MAX_UNK_TOKEN_RATIO` | スレのフィルタリング条件：`UNK`の最大比率 |
| `THREAD_FILTER_NG_PHRASES` | スレのフィルタリング条件： |

※スレタイなどが含まれる出力は削除しています。

In [1]:
mecab_dict_path = "/usr/share/mecab/dic/mecab-ipadic-neologd"

max_vocab=20000
THREAD_FILTER_TH_MIN_RESPONSE = 5
THREAD_FILTER_TH_MIN_TOKEN = 5
THREAD_FILTER_TH_MAX_TOKEN = 24  # 22 + SOS + EOS
THREAD_FILTER_TH_MAX_UNK_TOKEN_RATIO = 0.3
THREAD_FILTER_NG_PHRASES = [
    "ID:", "http"
]

In [2]:
import os
import re
import glob
import pickle

from utils.text_encoder import NanJThreadTitleEncoder

In [3]:
def load_titles(filepath_list):
    titles = []
    for fpath in filepath_list:
        with open(fpath) as f:
            for line in f:
                text = line.rstrip()
                elem = re.split(r'\t+', text)
                if len(elem) != 2:
                    continue
                thread_title, num_response = elem
                # NGフレーズが含まれる場合除外
                for phrase in THREAD_FILTER_NG_PHRASES:
                    if phrase in thread_title:
                        continue
                # スレ数が少ないものを除外
                if int(num_response) < THREAD_FILTER_TH_MIN_RESPONSE:
                    continue
                titles.append(thread_title)
    return titles

In [ ]:
# filepath_list = ["../data/1234.tsv"]
filepath_list = glob.glob("../data/*.tsv")
titles = load_titles(filepath_list)
print(len(titles))
print(titles[:3])

In [5]:
encoder = NanJThreadTitleEncoder(mecab_dict_path)
encoder.tokenize("彡(゚)(゚)「円周率は3ｗｗｗ」ワイ「🤔」 [無断転載禁止]©2ch.net")

['SOS',
 'NANJMIN',
 '「',
 '円周率',
 'は',
 'NUM',
 'KUSAxN',
 '」',
 'ワイ',
 '「',
 '」',
 'EOS']

In [6]:
tokens_list = [tuple(encoder.tokenize(t)) for t in titles]
tokens_list = list(set(tokens_list))  # 重複排除（Tokenレベルでの重複排除）
tokens_list = list(filter(lambda t: THREAD_FILTER_TH_MIN_TOKEN <= len(t) <= THREAD_FILTER_TH_MAX_TOKEN,
                          tokens_list))

In [ ]:
tokens_list[:3]

In [8]:
encoder.fit(tokens_list, max_vocab=max_vocab)

In [9]:
list(encoder._id_to_token.items())[:10]

[(0, 'PAD'),
 (1, 'SOS'),
 (2, 'EOS'),
 (3, 'UNK'),
 (4, 'の'),
 (5, 'に'),
 (6, 'が'),
 (7, 'で'),
 (8, '】'),
 (9, '【')]

In [10]:
# save encoder
if not os.path.exists("../model"):
    os.mkdir("../model")
encoder.save_to_file("../model/text_encoder")

In [11]:
# encode tokens into ids
ids = [tuple(encoder.encode(tokens)) for tokens in tokens_list]
ids = list(set(ids))  # 重複排除（idレベルでの重複排除）
ids = list(filter(lambda x: 1.0 * x.count("UNK") / len(x) <= THREAD_FILTER_TH_MAX_UNK_TOKEN_RATIO, ids))
ids = list(map(list, ids))

In [12]:
len(ids)

1613388

In [13]:
ids[:5]

[[1,
  9,
  1349,
  8,
  10,
  144,
  10,
  11293,
  20,
  29,
  128,
  4,
  1872,
  6,
  788,
  2277,
  6,
  971,
  119,
  17,
  17,
  2],
 [1, 228, 229, 32, 688, 1719, 5, 2076, 2],
 [1,
  94,
  4,
  613,
  1042,
  323,
  491,
  288,
  7,
  185,
  4,
  10,
  436,
  504,
  5,
  68,
  56,
  18,
  933,
  2],
 [1, 1605, 11, 6147, 16262, 24, 103, 17, 2],
 [1, 3046, 12, 7300, 16, 10, 217, 5, 691, 26, 3, 22, 12448, 34, 13, 2]]

In [14]:
# save ids
with open("../model/input.pickle", mode="wb") as f:
    pickle.dump(ids, f, protocol=pickle.HIGHEST_PROTOCOL)